In [1]:
### parameters
place = 'tel_aviv'
feature = 'pop_dens'

In [2]:

import geopandas as gpd
from shapely.geometry import box

import os
from pathlib import Path

import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'


# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [19]:

# --- Load Data ---
streets_gdf = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]
areas_all = gpd.read_file(f'{detail_folder}/stat_areas.shp')[['Shem_Yis_1', 'geometry', 'Pop_Total']]

# --- Compute Population Density ---
areas_all = areas_all[areas_all['Pop_Total'] > 0]
areas_all['density'] = areas_all['Pop_Total'] / areas_all.geometry.area

# --- Clip Population Areas to Bounding Box of Street Layer ---
minx, miny, maxx, maxy = streets_gdf.total_bounds
bounding_polygon = box(minx, miny, maxx, maxy)
bounding_gdf = gpd.GeoDataFrame(geometry=[bounding_polygon], crs=streets_gdf.crs)
areas_gdf = gpd.clip(areas_all, bounding_gdf)

# --- Spatial Intersection ---
intersected = gpd.overlay(streets_gdf, areas_gdf, how='intersection')
intersected['length'] = intersected.geometry.length

# --- Compute Density per Street ---
def compute_density(group):
    total_length = group['length'].sum()
    if total_length == 0:
        return -1
    elif len(group) == 1:
        return group.iloc[0]['density']
    else:
        return (group['density'] * group['length']).sum() / total_length

density_by_street = intersected.groupby('oidrechov').apply(compute_density).reset_index()
density_by_street.columns = ['oidrechov', 'estimated_pop_dens']

# --- Merge with Original Streets ---
streets_result = streets_gdf.merge(density_by_street, on='oidrechov', how='left')
streets_result['estimated_pop_dens'] = streets_result['estimated_pop_dens'].fillna(-1)

# --- Save Result ---
streets_result.to_file(f"{detail_folder}/pop_dens.shp")

